In [118]:
from openpyxl import load_workbook, Workbook

import re
import itertools
from datetime import datetime, timedelta

In [8]:
china_holidy_sheet = load_workbook('/home/zheng/Documents/1、QDII基金境外节假日暂停安排 - 2022' +
                                        '/1、QDII基金境外节假日暂停安排 - 2022/公元2022年日历(A股放假版).xlsx')['2022日历（周六、日连一起）']

In [54]:
china_holdays = []
holiday_color = china_holidy_sheet['b8'].fill.fgColor
for i in range(5, 62):
    for j in range(2, 25):
        cell = china_holidy_sheet.cell(i, j)
        if cell.fill.fgColor == holiday_color and cell.data_type == 'n':
            china_holdays.append(timedelta(days = (cell.value - 2)) + datetime(1900, 1, 1))
china_holdays.sort()

In [108]:
other_holiday_wb = load_workbook('/home/zheng/Downloads/2022年QDII基金暂停安排-202212.xlsx')

In [67]:
def get_next_work_day(day, next = 1):
    next_day = day
    counter = next
    while counter > 0:
        next_day += timedelta(days = 1)
        if next_day in china_holdays:
            continue
        counter -= 1
    return next_day

def get_previous_work_day(day, pre = 1):
    pre_day = day
    counter = pre
    while counter > 0:
        pre_day += timedelta(days = -1)
        if pre_day in china_holdays:
            continue
        counter -= 1
    return pre_day
    
print(get_previous_work_day(datetime(2022, 10, 10), pre = 2))
print(get_next_work_day(datetime(2022, 9, 30), next = 2))

2022-09-29 00:00:00
2022-10-11 00:00:00


In [122]:
def calc(wb, sheet_name, fund_name, is_hongkong = False):
    loc_map = {'东交所': '日本', '纽交所': '美国', '伦交所': '英国', '港交所': '香港'}

    ro = re.compile(r'(\d+)日（(.+)）')
    ro1 = re.compile(r'(\d+)月')
    sheet = wb[sheet_name]
    row = 4
    other_holiday = []
    while not sheet.cell(row, 2).value is None:
        month = int(ro1.match(sheet.cell(row, 2).value).groups()[0])
        day_info = sheet.cell(row, 3).value
        loc = sheet.cell(row, 4).value

        re_re = ro.match(day_info).groups()
        day = int(re_re[0])
        reason_list = re_re[1].split('、')
        loc_list = loc.split('、')

        if is_hongkong:
            other_holiday.append(([datetime(2022, month, day)], [re_re[1]], set(loc_list)))
        else:
            assert(len(reason_list) <= len(loc_list))
            if len(reason_list) < len(loc_list):
                assert(len(reason_list) == 1)
                for i in range(len(loc_list) - 1):
                    reason_list.append(reason_list[0])
            reason_list_with_country = []
            for reason, loc in zip(reason_list, loc_list):
                country = loc_map[loc]
                prefix_country = '' if reason.startswith(country) else country
                suffix_day = f'（{month}月{day}日）'
                reason_list_with_country.append(prefix_country + reason + suffix_day)
            
            other_holiday.append(([datetime(2022, month, day)], reason_list_with_country, set(loc_list)))

        row += 1

    for day_list, reason_list, loc_set in other_holiday:
        print(f"Read {day_list[0]}, {'、'.join(reason_list)}, {'、'.join(loc_set)}")

    merge_idx = 0
    while True:
        if merge_idx == len(other_holiday) - 1:
            break

        check_idx = merge_idx + 1
        day1 = other_holiday[merge_idx][0][0]
        day2 = other_holiday[check_idx][0][0]
        assert(day1 not in china_holdays)
        assert(day2 not in china_holdays)
        
        is_china_holiday_range = True
        check_day = day1 + timedelta(days = 1)
        while check_day < day2:
            if check_day not in china_holdays:
                is_china_holiday_range = False
                break
            check_day += timedelta(days = 1)

        if is_china_holiday_range:
            print(f"Merge {other_holiday[check_idx][0]} into {other_holiday[merge_idx][0]}")
            other_holiday[merge_idx][0].extend(other_holiday[check_idx][0])
            other_holiday[merge_idx][1].extend(other_holiday[check_idx][1])
            other_holiday[merge_idx][2].update(other_holiday[check_idx][2])
            del other_holiday[check_idx]
        else:
            merge_idx += 1

    rtn = []    
    for day_list, reason_list, loc_set in other_holiday:
        first_day = day_list[0]
        last_day = day_list[-1]
        
        start_day = first_day
        end_day = get_next_work_day(last_day)
        publish_day = get_previous_work_day(first_day, pre = 5)
        submit_day = get_previous_work_day(first_day, pre = 6)
        
        if is_hongkong:
            reason_str = '；'.join(set(reason_list))
        else:
            reason_str = '、'.join(reason_list)

        loc_str = '、'.join(loc_set)
        rtn.append((fund_name, start_day, end_day, publish_day, submit_day, reason_str, loc_str))
    return rtn

In [124]:
hongkong = calc(other_holiday_wb, '1、南方香港', '南方香港')
oil = calc(other_holiday_wb, '2、南方原油', '南方原油')
reit = calc(other_holiday_wb, '3、美国REIT', '美国REIT')
sheet16 = calc(other_holiday_wb, 'Sheet16', '', True)

Read 2022-04-15 00:00:00, 香港耶稣受难日（4月15日）, 港交所
Read 2022-04-18 00:00:00, 香港复活节假期（4月18日）, 港交所
Read 2022-05-09 00:00:00, 香港佛诞日翌日（5月9日）, 港交所
Read 2022-07-01 00:00:00, 香港特别行政区成立纪念日（7月1日）, 港交所
Read 2022-12-26 00:00:00, 香港圣诞节假期（12月26日）, 港交所
Read 2022-12-27 00:00:00, 香港圣诞节假期（12月27日）, 港交所
Merge [datetime.datetime(2022, 4, 18, 0, 0)] into [datetime.datetime(2022, 4, 15, 0, 0)]
Merge [datetime.datetime(2022, 12, 27, 0, 0)] into [datetime.datetime(2022, 12, 26, 0, 0)]
Read 2022-01-10 00:00:00, 日本成人节（1月10日）, 东交所
Read 2022-01-17 00:00:00, 美国马丁•路德•金节（1月17日）, 纽交所
Read 2022-02-11 00:00:00, 日本建国纪念日（2月11日）, 东交所
Read 2022-02-21 00:00:00, 美国华盛顿诞辰日（2月21日）, 纽交所
Read 2022-02-23 00:00:00, 日本天皇诞辰（2月23日）, 东交所
Read 2022-03-21 00:00:00, 日本春分节（3月21日）, 东交所
Read 2022-04-15 00:00:00, 美国耶稣受难日（4月15日）、英国耶稣受难日（4月15日）, 纽交所、伦交所
Read 2022-04-18 00:00:00, 英国复活节假期（4月18日）, 伦交所
Read 2022-04-29 00:00:00, 日本昭和日（4月29日）, 东交所
Read 2022-05-05 00:00:00, 日本儿童节（5月5日）, 东交所
Read 2022-05-30 00:00:00, 美国阵亡将士纪念日（5月30日）, 纽交所
Read 2022-06-02 00

In [125]:
out_wb = Workbook()
out_sheet = out_wb.active
out_sheet.cell(1, 1).value = '基金'
out_sheet.cell(1, 2).value = '暂停起始日期'
out_sheet.cell(1, 3).value = '恢复日期'
out_sheet.cell(1, 4).value = '公告日期'
out_sheet.cell(1, 5).value = '提交日期'
out_sheet.cell(1, 6).value = '暂停理由'
out_sheet.cell(1, 7).value = '备注'
out_sheet.cell(1, 8).value = '完成情况 '


for idx, info in enumerate(itertools.chain(hongkong, oil, reit, sheet16)):
    row = idx + 2
    out_sheet.cell(row, 1).value = info[0]
    out_sheet.cell(row, 2).value = info[1].strftime('%Y/%m/%d')
    out_sheet.cell(row, 3).value = info[2].strftime('%Y/%m/%d')
    out_sheet.cell(row, 4).value = info[3].strftime('%Y/%m/%d')
    out_sheet.cell(row, 5).value = info[4].strftime('%Y/%m/%d')
    out_sheet.cell(row, 6).value = info[5]
    out_sheet.cell(row, 7).value = info[6]

out_wb.save(filename='/home/zheng/Downloads/2022年QDII及港股通暂停安排表 .xlsx')

In [30]:
china_holidy_sheet['b8'].fill.fgColor == china_holidy_sheet['b7'].fill.fgColor

True

In [51]:
china_holidy_sheet['b7'].data_type

'n'

In [57]:
china_holdays

[datetime.datetime(2022, 1, 1, 0, 0),
 datetime.datetime(2022, 1, 2, 0, 0),
 datetime.datetime(2022, 1, 3, 0, 0),
 datetime.datetime(2022, 1, 8, 0, 0),
 datetime.datetime(2022, 1, 9, 0, 0),
 datetime.datetime(2022, 1, 15, 0, 0),
 datetime.datetime(2022, 1, 16, 0, 0),
 datetime.datetime(2022, 1, 22, 0, 0),
 datetime.datetime(2022, 1, 23, 0, 0),
 datetime.datetime(2022, 1, 29, 0, 0),
 datetime.datetime(2022, 1, 30, 0, 0),
 datetime.datetime(2022, 1, 31, 0, 0),
 datetime.datetime(2022, 2, 1, 0, 0),
 datetime.datetime(2022, 2, 2, 0, 0),
 datetime.datetime(2022, 2, 3, 0, 0),
 datetime.datetime(2022, 2, 4, 0, 0),
 datetime.datetime(2022, 2, 5, 0, 0),
 datetime.datetime(2022, 2, 6, 0, 0),
 datetime.datetime(2022, 2, 12, 0, 0),
 datetime.datetime(2022, 2, 13, 0, 0),
 datetime.datetime(2022, 2, 19, 0, 0),
 datetime.datetime(2022, 2, 20, 0, 0),
 datetime.datetime(2022, 2, 26, 0, 0),
 datetime.datetime(2022, 2, 27, 0, 0),
 datetime.datetime(2022, 3, 5, 0, 0),
 datetime.datetime(2022, 3, 6, 0, 0),